## Automate Business Metric Reporting with DataCamp Workspace

You can consult the solution by visiting `notebook-solution.ipynb` in the file browser.

For this code to work, you will need to set up a Slack app with the `chat:write`, `chat:write.customize`, `files:write` and `files:read` permissions, and store the OAuth token as an environment variable named `SLACK_OAUTH_TOKEN`.

In [1]:
!pip install slack_sdk
from slack_sdk import WebClient
import plotly.express as px
import os

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.5/276.5 kB 27.6 MB/s eta 0:00:00


## Total sales by month

In [2]:
SELECT 
    DATEFROMPARTS(DATEPART(year, o.order_date), DATEPART(month, o.order_date), 1) AS month,
    SUM(quantity * list_price * (1 - discount)) AS total_sales
FROM
    sales.orders AS o
    INNER JOIN sales.order_items AS oi ON o.order_id = oi.order_id
WHERE 
    o.order_date < '2018-05-01'
GROUP BY 
    DATEFROMPARTS(DATEPART(year, o.order_date), DATEPART(month, o.order_date), 1)
ORDER BY 
    month

,month,total_sales
0,2016-01-01 00:00:00+00:00,215146.4241
1,2016-02-01 00:00:00+00:00,156112.3228
2,2016-03-01 00:00:00+00:00,180600.3285
3,2016-04-01 00:00:00+00:00,167144.0512
4,2016-05-01 00:00:00+00:00,205270.0091
5,2016-06-01 00:00:00+00:00,210562.1245
6,2016-07-01 00:00:00+00:00,199556.8089
7,2016-08-01 00:00:00+00:00,225657.3767
8,2016-09-01 00:00:00+00:00,273091.6097
9,2016-10-01 00:00:00+00:00,212078.0805


In [3]:
fig = px.line(sales_over_time, x='month', y='total_sales', title='Total Sales by Month')
fig.show()
sales_chart_filename = 'sales_over_time.png'
fig.write_image(sales_chart_filename)

NameError: name 'px' is not defined

## Surface top sellers

In [9]:
SELECT TOP 3
	CONCAT(first_name, ' ', last_name) AS full_name,
	SUM(quantity * list_price * (1 - discount)) AS sales
FROM 
	sales.orders AS o
	INNER JOIN sales.order_items AS oi ON o.order_id = oi.order_id
	INNER JOIN sales.staffs AS s ON o.staff_id = s.staff_id
WHERE 
	DATEPART(year, o.order_date) = 2018
	AND DATEPART(month, o.order_date) = 4
GROUP BY 
	CONCAT(first_name, ' ', last_name)
ORDER BY 
	sales DESC

,full_name,sales
0,Venita Daniel,275594.3638
1,Genna Serrano,174986.5221
2,Marcelene Boyer,161670.5474


## Report metrics to Slack

In [13]:
from babel.numbers import format_currency

total_sales = sales_over_time[sales_over_time['month'] == sales_over_time['month'].max()]['total_sales'].values[0]
sellers_dict = top_sellers.to_dict(orient='records')

sellers_list = top_sellers \
    .apply(lambda x: f"• {x['full_name']} ({format_currency(x['sales'], 'USD')})", axis="columns")
sellers_overview = "\n".join(sellers_list)

text = f"""The past month we did {format_currency(total_sales, 'USD')} in sales!

Our sales all stars:
{sellers_overview}

_<https://app.datacamp.com/workspace/w/413c722c-cb2a-4fd6-9768-bd067f5764da/edit|Link to workspace>_
"""

channel = "sales-tracking"
if os.environ.get('DC_SCHEDULED_RUN') == 'TRUE':
    client = WebClient(token=os.environ["SLACK_OAUTH_TOKEN"])
    client.chat_postMessage(channel=channel, text=text)
    client.files_upload(channels=channel, file=sales_chart_filename)
else:
    print(text)

NameError: name 'sales_over_time' is not defined